In [1]:
#Installing the necessary libraries
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://bitbucket.org/habedi/datasets/raw/b6769c4664e7ff68b001e2f43bc517888cbe3642/spark/spark-3.0.2-bin-hadoop2.7.tgz
!tar xf spark-3.0.2-bin-hadoop2.7.tgz
!rm -rf spark-3.0.2-bin-hadoop2.7.tgz*
!pip -q install findspark pyspark graphframes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 13.2 MB/s eta 0:00:00


In [2]:
#Initializing the environment
import os
os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.1-spark3.0-s_2.12 pyspark-shell"

In [3]:
from graphframes import *
from pyspark import *
from pyspark.sql import *
spark = SparkSession.builder.appName('proj').getOrCreate()

<h2><b>Importing datasets:<h2/>
We are going to be working with two datasets in the first part of our project.The nodes dataset maps the names of 500 individuals whereas the edges dataset helps us understand the relation between source and destination column.

In [4]:
vertices = spark.read.option('header', 'true').csv('nodes.csv')
edges = spark.read.option('header', 'true').csv('edges.csv')

In [5]:
vertices.count()

500

In [6]:
edges.count()

1500

In [7]:
#Creating a graph of vertices and edges
graph = GraphFrame(vertices, edges)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


**GraphFrame Analysis:**
By creating a graphFrame we will be able to understand the customers who buys from the highest number of workers which is shown by the id having the highest outdegree.Similarly we are using the pageRank method of GraphFrame to analyze which userid was was the most frequent id in the whole dataset.By completing further analysis on this data we can also find out

In [8]:
#Displaying the vertices of graph
graph.vertices.show()

#Displaying the edges of graph
graph.edges.show()

+---+------+
| id|  name|
+---+------+
|  1|worker|
|  2|worker|
|  3|worker|
|  4|worker|
|  5|worker|
|  6|worker|
|  7|worker|
|  8|worker|
|  9|worker|
| 10|worker|
| 11|worker|
| 12|worker|
| 13|worker|
| 14|worker|
| 15|worker|
| 16|worker|
| 17|worker|
| 18|worker|
| 19|worker|
| 20|worker|
+---+------+
only showing top 20 rows

+---+---+--------+-------------+------+
|src|dst|relation|         item|rating|
+---+---+--------+-------------+------+
|257| 57|    buys|         pens|     2|
|210|151|    buys|   headphones|     1|
|395|158|    buys|       laptop|     5|
|160| 73|    buys|   headphones|     3|
|357| 55|    buys|        mouse|     2|
| 48| 52| manages|       online|    10|
|350|112|    buys|    usb cable|     2|
| 65|166|    buys|        mouse|     5|
|241|116|    buys|  paper clips|     2|
|239|100|    buys|      pencils|     2|
|133| 63|    buys|      monitor|     3|
|402| 69|    buys|         pens|     1|
|466| 53|    buys|        mouse|     3|
|353| 59|    buys|    

In [10]:
#Displaying the users and number of out degress
graph.outDegrees.show()

#Displaying top-5 users having maximum outDegrees
graph.outDegrees.orderBy(['outDegree'], ascending = [False]).show()

#Displaying bottom-3 users having minimum outDegrees
graph.inDegrees.orderBy(['inDegree'], ascending = [False]).show()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.
  warnings.warn("DataFrame constructor is internal. Do not directly use it.")


+---+---------+
| id|outDegree|
+---+---------+
|467|        5|
|296|        1|
|451|        7|
|125|        3|
|  7|        3|
| 51|        3|
|124|        2|
|447|        1|
|307|        3|
|475|        1|
|334|       10|
|205|        4|
|169|        5|
|462|        4|
|442|        1|
|470|        1|
|272|        1|
| 54|        2|
|232|        2|
|282|        3|
+---+---------+
only showing top 20 rows

+---+---------+
| id|outDegree|
+---+---------+
|334|       10|
|401|       10|
|188|        9|
|160|        9|
| 98|        8|
| 71|        8|
| 78|        8|
|427|        8|
|305|        8|
|298|        7|
|390|        7|
|451|        7|
|166|        7|
|433|        7|
| 89|        7|
|229|        7|
| 55|        7|
|322|        7|
| 88|        7|
| 37|        7|
+---+---------+
only showing top 20 rows

+---+--------+
| id|inDegree|
+---+--------+
|125|      23|
|116|      22|
| 91|      21|
|137|      21|
| 63|      19|
|121|      19|
|151|      19|
|112|      18|
|171|      18|


In [11]:
y=graph.pageRank(maxIter=10)
y.vertices.show()

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


+---+------+-------------------+
| id|  name|           pagerank|
+---+------+-------------------+
|  1|worker|0.18298212506590952|
|  2|worker|0.18298212506590952|
|  3|worker|0.18298212506590952|
|  4|worker|0.18298212506590952|
|  5|worker|0.18298212506590952|
|  6|worker|0.18298212506590952|
|  7|worker|0.18298212506590952|
|  8|worker|0.18298212506590952|
|  9|worker|0.18298212506590952|
| 10|worker|0.18298212506590952|
| 11|worker|0.18298212506590952|
| 12|worker|0.18298212506590952|
| 13|worker|0.18298212506590952|
| 14|worker|0.18298212506590952|
| 15|worker|0.18298212506590952|
| 16|worker|0.18298212506590952|
| 17|worker|0.18298212506590952|
| 18|worker|0.18298212506590952|
| 19|worker|0.18298212506590952|
| 20|worker|0.18298212506590952|
+---+------+-------------------+
only showing top 20 rows



In [14]:
y_vertices = y.vertices                                                  #Grouping the edges and vertices
data_edges = graph.edges.select("src", "dst", "relation","item","rating")
y_with_relations = GraphFrame(y_vertices, data_edges)
y_with_relations.vertices.show()


+---+------+-------------------+
| id|  name|           pagerank|
+---+------+-------------------+
|  1|worker|0.18298212506590952|
|  2|worker|0.18298212506590952|
|  3|worker|0.18298212506590952|
|  4|worker|0.18298212506590952|
|  5|worker|0.18298212506590952|
|  6|worker|0.18298212506590952|
|  7|worker|0.18298212506590952|
|  8|worker|0.18298212506590952|
|  9|worker|0.18298212506590952|
| 10|worker|0.18298212506590952|
| 11|worker|0.18298212506590952|
| 12|worker|0.18298212506590952|
| 13|worker|0.18298212506590952|
| 14|worker|0.18298212506590952|
| 15|worker|0.18298212506590952|
| 16|worker|0.18298212506590952|
| 17|worker|0.18298212506590952|
| 18|worker|0.18298212506590952|
| 19|worker|0.18298212506590952|
| 20|worker|0.18298212506590952|
+---+------+-------------------+
only showing top 20 rows



In [15]:
type(vertices)

pyspark.sql.dataframe.DataFrame

In [16]:
data_vertices = y_with_relations.vertices.orderBy("pagerank", ascending=False)
data_vertices.show()

+---+------+------------------+
| id|  name|          pagerank|
+---+------+------------------+
|116|worker|14.036168656614949|
|140|worker|13.697565136254159|
|148|worker|11.361609807300171|
|150|worker|11.358469492346893|
|112|worker| 9.314862111594387|
| 60|worker| 8.934199749365282|
|145|worker| 8.630717578208252|
| 90|worker| 8.269886818609974|
| 82|worker| 7.978091233052873|
|137|worker|  7.96861407927161|
|135|worker| 7.885009739966228|
| 91|worker|6.9300302284726865|
| 97|worker|  6.70673022652916|
|163|worker| 6.644861902845732|
|141|worker| 6.533074969978541|
| 69|worker| 6.257346013964667|
| 95|worker| 6.246651819386722|
| 58|worker| 6.068394813226532|
|171|worker| 5.961323098361945|
|100|worker| 5.931271079472601|
+---+------+------------------+
only showing top 20 rows



In [17]:
type(data_vertices)

pyspark.sql.dataframe.DataFrame

In [18]:
data_edges=y_with_relations.edges
data_edges.show()

+---+---+--------+-------------+------+
|src|dst|relation|         item|rating|
+---+---+--------+-------------+------+
|257| 57|    buys|         pens|     2|
|210|151|    buys|   headphones|     1|
|395|158|    buys|       laptop|     5|
|160| 73|    buys|   headphones|     3|
|357| 55|    buys|        mouse|     2|
| 48| 52| manages|       online|    10|
|350|112|    buys|    usb cable|     2|
| 65|166|    buys|        mouse|     5|
|241|116|    buys|  paper clips|     2|
|239|100|    buys|      pencils|     2|
|133| 63|    buys|      monitor|     3|
|402| 69|    buys|         pens|     1|
|466| 53|    buys|        mouse|     3|
|353| 59|    buys|         pens|     4|
|132|137|    buys|paper weights|     5|
|375|158|    buys|    mouse pad|     1|
|133|173|    buys|       laptop|     5|
| 98|147|    buys|       laptop|     5|
|107| 57|    buys|paper weights|     2|
|413| 52|    buys|       laptop|     2|
+---+---+--------+-------------+------+
only showing top 20 rows



Inorder to process our data further, that is joining the vertices and edges to form a combined sql dataframe, "join" is used.

In [20]:
data1=data_vertices.join(data_edges, data_vertices["id"].startswith(data_edges["src"]),"full_outer")
data1.show(3)

+---+------+-------------------+---+---+--------+-------+------+
| id|  name|           pagerank|src|dst|relation|   item|rating|
+---+------+-------------------+---+---+--------+-------+------+
|  1|worker|0.18298212506590952|  1|168| manages| online|    30|
|  2|worker|0.18298212506590952|  2|114| manages|offline|    30|
|  2|worker|0.18298212506590952|  2| 76| manages|offline|    40|
+---+------+-------------------+---+---+--------+-------+------+
only showing top 3 rows



In [21]:
data2=data1.orderBy("pagerank", ascending=False)
data2.show(3)

+---+------+------------------+---+---+--------+-------+------+
| id|  name|          pagerank|src|dst|relation|   item|rating|
+---+------+------------------+---+---+--------+-------+------+
|116|worker|14.036168656614949|116| 51|    buys|pencils|     4|
|116|worker|14.036168656614949|  1|168| manages| online|    30|
|116|worker|14.036168656614949|116|145|    buys| laptop|     3|
+---+------+------------------+---+---+--------+-------+------+
only showing top 3 rows



In [22]:
data_2 = data2.limit(1000)                               #Filtering top 1000 records

In [23]:
final=data_2.filter(data2['relation']=="buys")            # Filtering the data based on the relations

In [24]:
final2=data_2.filter(data2['relation']=="manages")

In [25]:
type(final)

pyspark.sql.dataframe.DataFrame

In [26]:
final_data=final.dropna()

In [27]:
final_data1=final2.dropna()

In [28]:
cluster_data=final_data.select("name","src","dst","pagerank","item","rating")

In [30]:
cluster_data.show(3)

+------+---+---+------------------+---------+------+
|  name|src|dst|          pagerank|     item|rating|
+------+---+---+------------------+---------+------+
|worker|116| 51|14.036168656614949|  pencils|     4|
|worker|116|145|14.036168656614949|   laptop|     3|
|worker|116|134|14.036168656614949|mouse pad|     3|
+------+---+---+------------------+---------+------+
only showing top 3 rows



In [31]:
cluster_data2=final_data1.select("name","src","dst","pagerank","item","rating")

In [32]:
cluster_data2.show(3)

+------+---+---+------------------+-------+------+
|  name|src|dst|          pagerank|   item|rating|
+------+---+---+------------------+-------+------+
|worker|  1|168|14.036168656614949| online|    30|
|worker| 14|107|13.697565136254159|offline|    40|
|worker| 14| 81|13.697565136254159|offline|    20|
+------+---+---+------------------+-------+------+
only showing top 3 rows



In [33]:
from pyspark.ml.feature import VectorAssembler
assembler=VectorAssembler(inputCols=['pagerank','rating_int'], outputCol='all_features')

In [34]:
cluster_data.dtypes

[('name', 'string'),
 ('src', 'string'),
 ('dst', 'string'),
 ('pagerank', 'double'),
 ('item', 'string'),
 ('rating', 'string')]

For further analysis into a the vector assembler, all the included column must be of the type int.

In [35]:
from pyspark.sql.functions import col                                            #converting rating string to int
cluster1_data = cluster_data.withColumn('rating_int', col('rating').cast('int'))
cluster1_data=cluster1_data.drop("rating")

In [36]:
cluster_data3=cluster_data2.withColumn('rating_int', col('rating').cast('int'))
cluster_data3=cluster_data3.drop("rating")

In [37]:
cluster_data3.show(3)

+------+---+---+------------------+-------+----------+
|  name|src|dst|          pagerank|   item|rating_int|
+------+---+---+------------------+-------+----------+
|worker|  1|168|14.036168656614949| online|        30|
|worker| 14|107|13.697565136254159|offline|        40|
|worker| 14| 81|13.697565136254159|offline|        20|
+------+---+---+------------------+-------+----------+
only showing top 3 rows



In [38]:
cluster1_data.show(3)

+------+---+---+------------------+---------+----------+
|  name|src|dst|          pagerank|     item|rating_int|
+------+---+---+------------------+---------+----------+
|worker|116| 51|14.036168656614949|  pencils|         4|
|worker|116|145|14.036168656614949|   laptop|         3|
|worker|116|134|14.036168656614949|mouse pad|         3|
+------+---+---+------------------+---------+----------+
only showing top 3 rows



In [43]:
newdata=assembler.transform(cluster1_data)
newdata.show(3, truncate=False)

+------+---+---+------------------+---------+----------+------------------------+
|name  |src|dst|pagerank          |item     |rating_int|all_features            |
+------+---+---+------------------+---------+----------+------------------------+
|worker|116|51 |14.036168656614949|pencils  |4         |[14.036168656614949,4.0]|
|worker|116|145|14.036168656614949|laptop   |3         |[14.036168656614949,3.0]|
|worker|116|134|14.036168656614949|mouse pad|3         |[14.036168656614949,3.0]|
+------+---+---+------------------+---------+----------+------------------------+
only showing top 3 rows



In [42]:
newdata1=assembler.transform(cluster_data3)
newdata1.show(3, truncate = False)

+------+---+---+------------------+-------+----------+-------------------------+
|name  |src|dst|pagerank          |item   |rating_int|all_features             |
+------+---+---+------------------+-------+----------+-------------------------+
|worker|1  |168|14.036168656614949|online |30        |[14.036168656614949,30.0]|
|worker|14 |107|13.697565136254159|offline|40        |[13.697565136254159,40.0]|
|worker|14 |81 |13.697565136254159|offline|20        |[13.697565136254159,20.0]|
+------+---+---+------------------+-------+----------+-------------------------+
only showing top 3 rows



<h1><b>Clustering</h1>

In [44]:
from pyspark.ml.clustering import KMeans

In [46]:
kmeans=KMeans(featuresCol="all_features").setK(5).setSeed(10)
results=kmeans.fit(newdata).transform(newdata)
results.show(3, truncate = False)

+------+---+---+------------------+---------+----------+------------------------+----------+
|name  |src|dst|pagerank          |item     |rating_int|all_features            |prediction|
+------+---+---+------------------+---------+----------+------------------------+----------+
|worker|116|51 |14.036168656614949|pencils  |4         |[14.036168656614949,4.0]|4         |
|worker|116|145|14.036168656614949|laptop   |3         |[14.036168656614949,3.0]|4         |
|worker|116|134|14.036168656614949|mouse pad|3         |[14.036168656614949,3.0]|4         |
+------+---+---+------------------+---------+----------+------------------------+----------+
only showing top 3 rows



In [47]:
results1=kmeans.fit(newdata1).transform(newdata1)
results1.show(3, truncate=False)

+------+---+---+------------------+-------+----------+-------------------------+----------+
|name  |src|dst|pagerank          |item   |rating_int|all_features             |prediction|
+------+---+---+------------------+-------+----------+-------------------------+----------+
|worker|1  |168|14.036168656614949|online |30        |[14.036168656614949,30.0]|4         |
|worker|14 |107|13.697565136254159|offline|40        |[13.697565136254159,40.0]|0         |
|worker|14 |81 |13.697565136254159|offline|20        |[13.697565136254159,20.0]|2         |
+------+---+---+------------------+-------+----------+-------------------------+----------+
only showing top 3 rows



In [50]:
cluster1=results1.filter(results1['prediction']==0)
print("Cluster 1 based on manages relation",cluster1.select('item').count())

cluster1_=results.filter(results['prediction']==0)
print("Cluster 1 based on buys relation",cluster1_.select('item').count())

Cluster 1 based on manages relation 70
Cluster 1 based on buys relation 143


In [51]:
cluster2=results1.filter(results1['prediction']==1)
print("Cluster 2 based on manages relation",cluster2.select('item').count())

cluster2_=results.filter(results['prediction']==1)
print("Cluster 2 based on buys relation",cluster2_.select('item').count())

Cluster 2 based on manages relation 116
Cluster 2 based on buys relation 122


In [52]:
cluster3=results1.filter(results1['prediction']==2)
print("Cluster 3 for manages relation",cluster3.select('item').count())

cluster3_=results.filter(results['prediction']==2)
print("Cluster 3 for buys relation",cluster3_.select('item').count())

Cluster 3 for manages relation 78
Cluster 3 for buys relation 24


In [53]:
cluster4=results1.filter(results1['prediction']==3)
print("Cluster 4 for manages relation",cluster4.select('item').count())

cluster4_=results.filter(results['prediction']==3)
print("Cluster 4 for buys relation",cluster4_.select('item').count())

Cluster 4 for manages relation 104
Cluster 4 for buys relation 98


In [54]:
cluster5=results1.filter(results1['prediction']==4)
print("Cluster 5 for manage relation",cluster5.select('item').count())

cluster5_=results.filter(results['prediction']==4)
print("Cluster 5 for buys relation",cluster5_.select('item').count())

Cluster 5 for manage relation 240
Cluster 5 for buys relation 5


**Inference:**
The distinct clusters suggest that there are different groups or categories within the dataset.Each cluster represents a set of data points with similar characteristics based on the features (Feature 1 and Feature 2).The alignment of clusters along the Feature 2 axis indicates that Feature 2 plays a significant role in defining these groups.Feature 1, on the other hand, does not show a clear pattern in clustering, but it still affects the density within each cluster.
The varying densities within each cluster imply that there might be subgroups within the identified clusters.Some clusters have higher concentrations of data points (yellow regions), while others have lower densities (purple regions).



In [55]:
#Plotting the kmeans clusters

import plotly.graph_objects as go
cluster_assignments = results1.select('prediction').rdd.map(lambda row: row[0]).collect()
features = newdata1.select('all_features').rdd.map(lambda row: row[0]).collect()
x_values = [feat[0] for feat in features]
y_values = [feat[1] for feat in features]

fig = go.Figure(data=go.Scattergl(x=x_values, y=y_values, mode='markers', marker=dict(color=cluster_assignments, colorscale='viridis')))

fig.update_layout(title='KMeans Plot(Manages relation)',
                  xaxis_title='Page Rank',
                  yaxis_title='Rating',
                  coloraxis=dict(colorscale='viridis', colorbar=dict(title='Cluster')))
fig.show()


In [56]:
# Plotting the kmeans clusters
import plotly.graph_objects as go

# Extracting cluster assignments
cluster_assignments = results.select('prediction').rdd.map(lambda row: row[0]).collect()

# Extracting features for plotting
features = newdata.select('all_features').rdd.map(lambda row: row[0]).collect()

# Unpacking features into separate lists for x and y axes
x_values = [feat[0] for feat in features]
y_values = [feat[1] for feat in features]

# Create scatter plot
fig = go.Figure(data=go.Scattergl(x=x_values, y=y_values, mode='markers', marker=dict(color=cluster_assignments, colorscale='viridis')))

# Update layout
fig.update_layout(title='KMeans Plot-buys relation',
                  xaxis_title='Page Rank',
                  yaxis_title='Rating',
                  coloraxis=dict(colorscale='viridis', colorbar=dict(title='Cluster')))

# Show plot
fig.show()


**Inference:**The purple cluster should mainly consist of customers who have placed very few orders from workers.As we progress towards the roght in the graph the number of customers will start reducing in the green cluster and number of workers in the cluster will increase.This will keep increasing as we keep going right and the blue and yellow cluster should predominantly consists of workers but the amount of customers in this cluster will get reduced.

<h1><b>Collaborative Filtering</h1>

In [58]:
training=spark.read.csv("edges.csv", header=True, inferSchema=True)
training.show(3)
test=spark.read.csv("test.csv", header=True, inferSchema=True)
test.show(3)

+---+---+--------+----------+------+
|src|dst|relation|      item|rating|
+---+---+--------+----------+------+
|257| 57|    buys|      pens|     2|
|210|151|    buys|headphones|     1|
|395|158|    buys|    laptop|     5|
+---+---+--------+----------+------+
only showing top 3 rows

+---+------+
|src|  item|
+---+------+
|176|  pens|
|350| mouse|
|405|laptop|
+---+------+
only showing top 3 rows



In [59]:
from pyspark.ml.feature import StringIndexer
str_idx=StringIndexer(inputCols=['item'], outputCols=['item_ID'], stringOrderType='alphabetAsc')
newtraining=str_idx.fit(training).transform(training)
newtraining.show(3)
newtest=str_idx.fit(test).transform(test)
newtest.show(3)

+---+---+--------+----------+------+-------+
|src|dst|relation|      item|rating|item_ID|
+---+---+--------+----------+------+-------+
|257| 57|    buys|      pens|     2|   10.0|
|210|151|    buys|headphones|     1|    0.0|
|395|158|    buys|    laptop|     5|    1.0|
+---+---+--------+----------+------+-------+
only showing top 3 rows

+---+------+-------+
|src|  item|item_ID|
+---+------+-------+
|176|  pens|    8.0|
|350| mouse|    3.0|
|405|laptop|    1.0|
+---+------+-------+
only showing top 3 rows



In [61]:
#Creating recommendation system using alternating least squares
from pyspark.ml.recommendation import ALS
model=ALS(userCol='src', itemCol='item_ID', ratingCol='rating')

newmodel=model.fit(newtraining)
results=newmodel.transform(newtest)
results.show(3)

+---+---------+-------+----------+
|src|     item|item_ID|prediction|
+---+---------+-------+----------+
|405|   laptop|    1.0| 2.8673477|
|350|    mouse|    3.0| 1.7820501|
|450|mouse pad|    4.0| 0.6581031|
+---+---------+-------+----------+
only showing top 3 rows



<p>Utilizing collaborative filtering techniques enabled accurate prediction of customer preferences for items they hadn't previously purchased, facilitating personalized recommendations and enhancing the shopping experience.

</p>

<h1><b>Frequent Pattern Mining</h1>

In [63]:
#Preprocessing
fpm1=spark.read.csv("items.csv", header=True, inferSchema=True)
fpm1.show(3, truncate=False)

+-------------------------------+
|item                           |
+-------------------------------+
|usb cable;paper weights;pencils|
|paper clips;laptop;mouse pad   |
|mouse;pencils;pens             |
+-------------------------------+
only showing top 3 rows



In [64]:
type(fpm1)

pyspark.sql.dataframe.DataFrame

In [65]:
print("Number of observations in dataset:",fpm1.count())
print("First few records:\n",fpm1.show(3,truncate=False))

Number of observations in dataset: 1750
+-------------------------------+
|item                           |
+-------------------------------+
|usb cable;paper weights;pencils|
|paper clips;laptop;mouse pad   |
|mouse;pencils;pens             |
+-------------------------------+
only showing top 3 rows

First few records:
 None


In [66]:
fpm1.dtypes

[('item', 'string')]

For further analysis into the Market Basket Analysis, the FPGrowth algorithms requires a list column.

In [71]:
fpm2 = fpm1.select('item')

In [72]:
#Converting column into array
from pyspark.sql.functions import split, col
fpm3=fpm2.withColumn("item", split(col("item"), ";"))

In [73]:
fpm3.show(3,truncate=False)

+-----------------------------------+
|item                               |
+-----------------------------------+
|[usb cable, paper weights, pencils]|
|[paper clips, laptop, mouse pad]   |
|[mouse, pencils, pens]             |
+-----------------------------------+
only showing top 3 rows



In [75]:
#Importing necessary libraries and creating an object
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="item", minSupport=0.0001, minConfidence=0.0004)
model = fpGrowth.fit(fpm3)
itemsets=model.freqItemsets
print("Number of observations in new data set:",itemsets.count())
print(itemsets.show(3,truncate=False))

Number of observations in new data set: 231
+--------------------------------+----+
|items                           |freq|
+--------------------------------+----+
|[paper weights]                 |460 |
|[paper weights, mouse]          |89  |
|[paper weights, mouse, stickers]|10  |
+--------------------------------+----+
only showing top 3 rows

None


In [80]:
# Display generated association rules.
rules=model.associationRules
rules.show(3,truncate=False)

+---------------------+---------------+-------------------+-------------------+--------------------+
|antecedent           |consequent     |confidence         |lift               |support             |
+---------------------+---------------+-------------------+-------------------+--------------------+
|[headphones, pencils]|[monitor]      |0.06153846153846154|0.33444816053511706|0.002285714285714286|
|[headphones, pencils]|[paper weights]|0.13846153846153847|0.5267558528428095 |0.005142857142857143|
|[headphones, pencils]|[usb cable]    |0.06153846153846154|0.23360587351910567|0.002285714285714286|
+---------------------+---------------+-------------------+-------------------+--------------------+
only showing top 3 rows



Thershold lift is 1.1

In [77]:
rules1=rules.filter(rules['lift']>1.1)
rules1.orderBy("support", ascending=False).show(3)

+-------------+-------------+-------------------+------------------+-------------------+
|   antecedent|   consequent|         confidence|              lift|            support|
+-------------+-------------+-------------------+------------------+-------------------+
|[paper clips]|   [stickers]|0.33816425120772947|1.1883281919950333|               0.12|
|   [stickers]|[paper clips]|0.42168674698795183|1.1883281919950333|               0.12|
|      [mouse]|  [mouse pad]| 0.3882113821138211|1.4454679121259297|0.10914285714285714|
+-------------+-------------+-------------------+------------------+-------------------+
only showing top 3 rows



Confidence threshold=0.2 and support threshold=0.1

In [78]:
rules2=rules.filter((rules['confidence']>0.2) | (rules['support']>0.1))
rules2.orderBy("support",ascending=False).show(3)

+-------------+-------------+-------------------+------------------+-------------------+
|   antecedent|   consequent|         confidence|              lift|            support|
+-------------+-------------+-------------------+------------------+-------------------+
|[paper clips]|   [stickers]|0.33816425120772947|1.1883281919950333|               0.12|
|   [stickers]|[paper clips]|0.42168674698795183|1.1883281919950333|               0.12|
|  [mouse pad]|      [mouse]|0.40638297872340423|1.4454679121259297|0.10914285714285714|
+-------------+-------------+-------------------+------------------+-------------------+
only showing top 3 rows



Confidence threshold=0.2 and lift threshold=0.1

In [79]:
rules3=rules.filter((rules['confidence']>0.2) & (rules['lift']>1.1))
rules3.orderBy("lift",ascending=False).show(3)

+-------------------+-----------+-------------------+------------------+--------------------+
|         antecedent| consequent|         confidence|              lift|             support|
+-------------------+-----------+-------------------+------------------+--------------------+
|[headphones, mouse]|[mouse pad]|0.40298507462686567|1.5004763416957765|0.015428571428571429|
|            [mouse]|[mouse pad]| 0.3882113821138211|1.4454679121259297| 0.10914285714285714|
|        [mouse pad]|    [mouse]|0.40638297872340423|1.4454679121259297| 0.10914285714285714|
+-------------------+-----------+-------------------+------------------+--------------------+
only showing top 3 rows

